###  Pipeline validation
#### Pipeline version v6.3.5

A comprehensive gene annotation (PRI) is used in this pipeline: https://www.gencodegenes.org/human/release_27.html

In [1]:
### Syntex validation ###
!java -jar ../../womtool/womtool-85.jar validate -l ../scAtlas/optimus/Optimus.wdl

Success!
List of Workflow dependencies is:
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/CheckInputs.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/FastqProcessing.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/RunEmptyDrops.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/cellbender_remove_background.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/StarAlign.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/H5adUtils.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/MergeSortBam.wdl
/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/tasks/Metrics.wdl
